# SCohenLab 2D Image Processing notebook (Simplified MCZ)

--------------
# PIPELINE OVERVIEW
## 1. GOAL SETTING

### GOAL:  Infer sub-cellular components in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components (Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and SOMA) during differentiation of iPSCs, in order to understand the Interactome / Spatiotemporal coordination.

### summary of _OBJECTIVES_
- Infer subcellular objects:
  -  #### #1. [infer NUCLEI ](#image-processing-objective-1-infer-nucleii) -
  -  #### #2. [Infer SOMA](#image-processing-objective-2-infer-soma) -
  -  #### #3. [Infer CYTOSOL](#image-processing-objective-3-infer-cytosol)- 
  -  #### #4. [Infer LYSOSOMES](#image-processing-objective-4-infer-lysosomes)  -
  -  #### #5. [Infer MITOCHONDRIA](#image-processing-objective-5-infer-mitochondria) - 
  -  #### #6. [Infer GOLGI complex](#image-processing-objective-6-infer-golgi-complex) - 
  -  #### #7. [Infer PEROXISOMES](#image-processing-objective-7-infer-peroxisomes) - 
  -  #### #8. [Infer ENDOPLASMIC RETICULUM ](#image-processing-objective-8-infer-endoplasmic-reticulum)-
  -  #### #9. [Infer LB](#image-processing-objective-9-infer-lipid-bodies-droplet)-





## 2. DATA CREATION
> METHODS
> iPSC lines prepared and visualized on Zeiss Microscopes. 32 channel multispectral images collected.  Linear Unmixing in  ZEN Blue software with target emission spectra yields 8 channel image outputs.  Channels correspond to: Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and a “residual” signal.

> Meta-DATA
>   - Microcope settings
>  - OME scheme
> - Experimenter observations
> - Sample, bio-replicate, image numbers, condition values, etc
>  - Dates
>  - File structure, naming conventions
>  - etc.





## 3. IMAGE PROCESSING
### INFERENCE OF SUB-CELLULAR OBJECTS
The imported images have already been pre-processed to transform the 32 channel spectral measuremnts into "linearly unmixed" images which estimate independently labeled sub-cellular components.  Thes 7 channels (plus a residual "non-linear" signal) will be used to infer the shapes and extents of these sub-cellular components.   
We will perform computational image analysis on the pictures (in 2D an 3D) to _segment_ the components of interest for measurement.  In other prcoedures we can used these labels as "ground truth" labels to train machine learning models to automatically perform the inference of these objects.
Pseudo-independent processing of the imported multi-channel image to acheive each of the 9 objecives stated above.  i.e. infering: NUCLEI, SOMA, CYTOSOL, LYSOSOME, MITOCHONDRIA, GOLGI COMPLEX, PEROZISOMES, ENDOPLASMIC RETICULUM, and LIPID BODIES

### General flow for infering objects via segmentation
- Pre-processing
- Core-processing (thresholding)
- Post-processing 

### QC




## 4. QUANTIFICATION

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



### NOTE: PIPELINE TOOL AND DESIGN CHOICES?
We want to leverage the Allen Cell & Structure Setmenter.  It has been wrapped as a [napari-plugin](https://www.napari-hub.org/plugins/napari-allencell-segmenter) but fore the workflow we are proving out here we will want to call the `aicssegmentation` [package](https://github.com/AllenCell/aics-segmentation) directly.

#### ​The Allen Cell & Structure Segmenter 
​The Allen Cell & Structure Segmenter is a Python-based open source toolkit developed at the Allen Institute for Cell Science for 3D segmentation of intracellular structures in fluorescence microscope images. This toolkit brings together classic image segmentation and iterative deep learning workflows first to generate initial high-quality 3D intracellular structure segmentations and then to easily curate these results to generate the ground truths for building robust and accurate deep learning models. The toolkit takes advantage of the high replicate 3D live cell image data collected at the Allen Institute for Cell Science of over 30 endogenous fluorescently tagged human induced pluripotent stem cell (hiPSC) lines. Each cell line represents a different intracellular structure with one or more distinct localization patterns within undifferentiated hiPS cells and hiPSC-derived cardiomyocytes.

More details about Segmenter can be found at https://allencell.org/segmenter
In order to leverage the A
# IMPORTS

import  all nescessary packages

we are using `napari` for visualization, and `scipy` `ndimage` and `skimage` for analyzing the image files.  The underlying data format are `numpy` `ndarrays` and tools from  Allen Institute for Cell Science.


In [4]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage import morphology
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import read_input_image, list_image_files, export_ome_tiff
from infer_subc_2d.utils.img import *

%load_ext autoreload
%autoreload 2

from infer_subc_2d.organelles import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


SyntaxError: invalid syntax (peroxisomes.py, line 2)


# Get and load Image for processing
For testing purposes... TODO: build a nice wrapper for this.



Read the data into memeory from the `.czi` files.  (Note: there is also the 2D slice .tif file read for later comparision).  WE will also collect metatdata.


In [4]:
# assumptions:   .czi "unmixed" collections.
#czi_img_folder = f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data"

data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")
czi_img_folder = data_path/"raw"

list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]

img_file_list = list_img_files(czi_img_folder,'.czi')
print(img_file_list[6])
#Read the data into memeory from the `.czi` files.  (Note: there is also the 2D slice .tif file read for later comparision).  WE will also collect metatdata.


/Users/ahenrie/Projects/Imaging/mcz_subcell/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N11_Unmixed.czi


In [5]:

test_img_name = img_file_list[6]

img_data, meta_dict = read_input_image(test_img_name)

raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']



/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


Visualize the raw data file with [napari](https://napari.org)

In [6]:
# # this does NOT seem to be in register with the segmentation !?!?!?

## TODO:  figure out why this "rotates" the axis.. .probably has to do with the implicit squeezing when extracting channels
# viewer = napari.view_image(
#     img_data,
#     channel_axis=0,
#     name=channel_names,
#     scale=scale
# )
# viewer.scale_bar.visible = True
scale

(0.5804527163320905, 0.07987165184837318, 0.07987165184837318)

In [ ]:
data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")

out_path = data_path / "inferred_objects" 
 
 
default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 10
})


################################

# calculate a filter dimension for median filtering which considers the difference in scale of Z
z_factor = scale[0]//scale[1]
med_filter_size = 4 #2D 
med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter
print(f"median filtering scale is ~ : { [x*y for x,y in zip(scale,med_filter_size_3D)]}")

default_params['z_factor'] = z_factor
default_params['scale'] = scale
# DEFAULT PARAMETERS:
#intensity_norm_param = [0.5, 15]
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100


Builde the segmentations in order




In [ ]:
# perform the segmentations in order to test we can build it up for one

i = 0
target_file = img_file_list[i]


img_data, meta_dict = read_input_image(target_file)
raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

raw_nuclei = img_data[0,:,:,:].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


In [ ]:
viewer = napari.view_image(
    raw_nuclei,
    scale=scale,
    colormap='blue', 
    blending='additive'
)
viewer.scale_bar.visible = True

viewer.add_image(
    NU_object,
    scale=scale,
    blending='additive'
)

viewer.dims.ndisplay = 3
viewer.camera.angles = (-30, 25, 120)


In [ ]:

###################
# SOMA INPUT
###################
raw_soma = (4. * img_data[1,:,:,:].copy() + 
                            1. * img_data[5,:,:,:].copy() + 
                            1. * img_data[7,:,:,:].copy() )

struct_img = intensity_normalization( raw_soma.copy() ,  scaling_param=scaling_param)
SO_object, SO_label, out_p =  infer_SOMA1(struct_img.copy(), NU_label, out_p) 

CY_object =  infer_CYTOSOL(SO_object, NU_object) 

# from here on we should probably _mask_ the objects... 
#   i.e. NU_object by SO_object or itself?

In [ ]:
viewer.add_image(
    raw_soma,
    scale=scale,
    blending='additive'
)


viewer.add_image(
    SO_object,
    scale=scale,
    blending='additive'
)

viewer.add_image(
    CY_object,
    scale=scale,
    blending='additive'
)


In [ ]:

viewer.dims.ndisplay = 2


In [ ]:

###################
# INPUT LY
###################
raw_lysosome = img_data[1,:,:,:].copy()


LY_object, out_p =  infer_LYSOSOMES(raw_lysosome.copy(), CY_object, out_p) 

###################
# INPUT MT
###################
raw_mito = img_data[2,:,:,:].copy()

MT_object, out_p =  infer_MITOCHONDRIA(raw_mito.copy(), CY_object, out_p) 



In [ ]:

###################
# INPUT GOLGI
###################
raw_golgi = img_data[3,:,:,:].copy()

GL_object, out_p =  infer_GOLGI(raw_golgi.copy(), CY_object, out_p) 



In [ ]:

###################
# INPUT PEROXISOME
###################
raw_peroxi = img_data[4,:,:,:].copy()

PO_object, out_p =  infer_PEROXISOME(raw_peroxi.copy(), CY_object, out_p) 


In [ ]:

###################
# INPUT ER
###################
raw_er = img_data[5,:,:,:].copy()

ER_object, out_p =  infer_ENDOPLASMIC_RETICULUM(raw_er.copy(), CY_object, out_p) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper


In [ ]:
viewer.add_image(
    raw_lysosome,
    scale=scale,
    colormap='cyan', 
    blending='additive'
)

viewer.add_image(
    LY_object,
    scale=scale,
    blending='additive'
)

viewer.add_image(
    raw_mito,
    scale=scale,
    colormap='green', 
    blending='additive'
)

viewer.add_image(
    MT_object,
    scale=scale,
    blending='additive'
)

viewer.add_image(
    raw_golgi,
    scale=scale,
    colormap='yellow', 
    blending='additive'
)

viewer.add_image(
    GL_object,
    scale=scale,
    blending='additive')

viewer.add_image(
    raw_er,
    scale=scale,
    colormap='red', 
    blending='additive'
)

viewer.add_image(
    ER_object,
    scale=scale,
    blending='additive')


In [ ]:

viewer.add_image(
    raw_peroxi,
    scale=scale,
    colormap='bop orange', 
    blending='additive'
)

viewer.add_image(
    PO_object,
    scale=scale,
    blending='additive')



Pack into an OME.tiff with the same 

In [ ]:
for target_file,i in enumemrate(img_file_list):


    img_data, meta_dict = read_input_image(test_imtarget_fileg_name)

    raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

    # get some top-level info about the RAW data
    channel_names = meta_dict['name']
    img = meta_dict['metadata']['aicsimage']
    scale = meta_dict['scale']
    channel_axis = meta_dict['channel_axis']

    raw_nuclei = img_data[0,:,:,:].copy()
    NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 

    ###################
    # INPUT
    ###################
    raw_soma = (4. * img_data[1,:,:,:].copy() + 
                                1. * img_data[5,:,:,:].copy() + 
                                1. * img_data[7,:,:,:].copy() )

    struct_img = intensity_normalization( raw_soma.copy() ,  scaling_param=scaling_param)
    SO_object, SO_label, out_p =  infer_SOMA1(struct_img.copy(), NU_labels, default_params) 

    CYTO_object =  infer_CYTOSOL(SO_object, NU_object) 


In [ ]:
##  need to save: 

# output_path, list_of_files

# TODO : wed Aug 24

- mask with SOMA
- test batches
- think about refactor
- speed improvement 
  - vectorize "slice-by-slice"

- metrics
  - start with `regeonprops`
